In [ ]:
from src.libs.lib import *
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
dfs = load_data_db()
dfs_original = dfs["drivers_occurences"]
df_original_table = pd.read_csv('src/data/tabela_de_pontuações.csv', delimiter=";")
df_original_table_correspondeces = pd.read_csv('src/data/tabela_de_pontuações_correspondencias.csv', delimiter=";")
columns = ['Código', 'Ocorrência DB-PX', 'Ocorrência Tabela - Academia',
       'Pontuação']
df_original_table_correspondeces.columns = columns


df = dfs_original.copy()

# Converter as colunas para datetime
df['contract_start'] = pd.to_datetime(df['contract_start'])
df['occurence_create'] = pd.to_datetime(df['occurence_create'])

# Calcular a diferença de data/hora em horas:
# A subtração entre as datas gera um objeto timedelta. Para converter para horas, usamos total_seconds()/3600.
df['diff_horas'] = (df['contract_start'] - df['occurence_create']).dt.total_seconds() / 3600
dfs_original= df

dfs_original = dfs_original.dropna(axis=0)

# Exibir o DataFrame resultante
display(df.head())
mask = df["description"] == "Desistência de contratos"
print("Proporção de desistencias antes do inicio do contrato")
print((df[mask]["diff_horas"] > 0).mean())

#5.5s

In [ ]:
print("df_original")
display(dfs_original.head())
print("Valores únicos")
display(pd.DataFrame(dfs_original["description"].unique()).sort_values(0))
print("df_original_table")
display(df_original_table)
print("df_original_table_correspondeces")
display(df_original_table_correspondeces)



In [ ]:
dfs_original = dfs_original.merge(df_original_table_correspondeces, how='left', left_on="description", right_on="Ocorrência DB-PX")
dfs_original.drop(["Código", "Ocorrência DB-PX", "Ocorrência Tabela - Academia"], axis=1, inplace=True)

In [ ]:
mask = (dfs_original["description"] == "Desistência de contratos") & (dfs_original["diff_horas"] >= 36)
dfs_original.loc[mask, "Pontuação"] = 0

mask = ((dfs_original["description"] == "Desistência de contratos") 
& (dfs_original["diff_horas"] < 36)
& (dfs_original["diff_horas"] >= 12))
dfs_original.loc[mask, "Pontuação"] = 125

mask = ((dfs_original["description"] == "Desistência de contratos") 
& (dfs_original["diff_horas"] < 12)
& (dfs_original["diff_horas"] >= 0))
dfs_original.loc[mask, "Pontuação"] = 185


#OCORRENCIAS CRIADAS APÓS O INICIO DO CONTRATO
mask = ((dfs_original["description"] == "Desistência de contratos") 
& (dfs_original["diff_horas"] < 0))
dfs_original.loc[mask, "Pontuação"] = 125


dfs_original = dfs_original.drop("diff_horas", axis=1) 

In [ ]:
mask = dfs_original["description"] == "Dia agenciado"
dfs_original.loc[mask, "Pontuação"] = -1*dfs_original.loc[mask, "observation"].astype(float)


In [ ]:
dfs_original.head(50)

In [ ]:
import pandas as pd

# Duplicate the original DataFrame
df = dfs_original.copy()

# Ensure numeric 'Pontuação' and datetime 'occurence_create'
df['Pontuação'] = pd.to_numeric(df['Pontuação'], errors='coerce').fillna(0)
df['occurence_create'] = pd.to_datetime(df['occurence_create'])

# Calculate first and last event dates per driver
first_dates = df.groupby('driver_id')['occurence_create'].min().rename('first_event')
last_dates = df.groupby('driver_id')['occurence_create'].max().rename('last_event')

# Calculate days elapsed between first and last event
dias_elapsed = (last_dates - first_dates).dt.days.rename('Dias decorridos') * (-1)

# Pivot for sum of Pontuação by driver_id and description
pivot_df = (
    df
    .pivot_table(
        index='driver_id',
        columns='description',
        values='Pontuação',
        aggfunc='sum',
        fill_value=0
    )
    .reset_index()
)
pivot_df.columns.name = None

# Merge the elapsed days into pivot_df
pivot_df = pivot_df.merge(dias_elapsed.reset_index(), on='driver_id')

# Display the result
pivot_df.head(2)


In [ ]:
# Recarregar os módulos após reset
import pandas as pd
import numpy as np

# Redefinir as colunas conforme fornecido
columns = ['Abandono do veículo', 'Aceleração brusca', 'Acidente',
           'Agressão física', 'Agressão verbal', 'Apropriação indevida', 'Avaria',
           'Banguela', 'Bebidas', 'Checklist Irregular', 'Cinto de segurança',
           'Comportamento', 'Conduta Motorista', 'Consumo médio',
           'Cuidado com equipamento/carga', 'Cumprimento de normas',
           'Desistência de contratos', 'Desvio de rota',
           'Distração ao volante', 'Distância Perigosa', 'Extravio de bens',
           'Extravio de carga', 'Fadiga na direção', 'Frenagem brusca',
           'Fumar ao volante', 'Imprudência', 'Não comparecimento em contrato',
           'Não retorna contato com a PX', 'Não retorna contato com cliente',
           'Outros', 'Parada não autorizada', 'Pontualidade', 'Prisão',
           'Problema de saúde', 'Relacionadas ao Motorista',
           'Relacionadas ao Transportador', 'Reprovado na Brasil Risk',
           'Reprovado na GR', 'Risco de tombamento', 'Sinistro',
           'Substâncias ilícitas', 'Uso do celular', 'Veículo', 'Dias decorridos', 'Dia agenciado']


# Calcular pontuação total
pivot_df["Pontuação"] = pivot_df[columns].sum(axis=1)

# Aplicar regras para Ação
dfs_original = pivot_df.copy()
dfs_original["Ação"] = "Nenhuma"
dfs_original.loc[(dfs_original["Pontuação"] > 370) & (dfs_original["Pontuação"] <= 740), "Ação"] = "Atualização"
dfs_original.loc[dfs_original["Pontuação"] > 740, "Ação"] = "Comitê"


In [ ]:
dfs_original.sort_values("Pontuação", ascending=False).head()

In [ ]:
dfs_original.to_csv("src/data/tabela_ocorrencias_dbpx_com_a_pontuação_academia.csv", index=False)
df_original_table_correspondeces.to_csv("src/data/tabela-dbpx-pontuação_academia.csv", index=False)


In [ ]:
import re
import unicodedata
import pandas as pd

def sanitize_column(col: str) -> str:
    # 1) Remover acentuação
    nfkd = unicodedata.normalize('NFKD', col)
    only_ascii = nfkd.encode('ASCII', 'ignore').decode('ASCII')
    # 2) Substituir qualquer coisa que não seja [A-Z, a-z, 0-9, _] por '_'
    clean = re.sub(r'[^0-9a-zA-Z_]', '_', only_ascii)
    # 3) Se começar com dígito, prefixar '_'
    if re.match(r'^[0-9]', clean):
        clean = '_' + clean
    # 4) Minúsculas
    clean = clean.lower()
    # 5) Truncar a 300 chars
    return clean[:300]

# Aplica o sanitize a **cada** coluna de df
dfs_original.columns = [sanitize_column(c) for c in dfs_original.columns]

# Agora df.columns tem o mesmo tamanho de antes, mas todos válidos para BigQuery
print(dfs_original.columns.tolist())


In [ ]:
texto = "12 - Peso das ocorrências comportamentais"
hash_5_digitos = gerar_hash_numerica(texto)
nome_tabela_bq = f"{hash_5_digitos}_pontuacao_ocorrencias_academia"
print(f"Hash numérica de 5 dígitos: {nome_tabela_bq}")
insert_into_bigquery(dfs_original, table_id=nome_tabela_bq)